In [3]:
import http.client
import json
from datetime import datetime
import pandas as pd
from rapidfuzz import process, fuzz
import psycopg2
import pandas as pd
from datetime import datetime, timedelta

In [9]:
# Load the CSV data into a DataFrame
airport_df = pd.read_csv('assets/airport_data.csv', usecols=['city', 'airport_code','railway_station_code'])

def get_closest_city(user_city):
    """
    Find the closest matching city to the user's input using fuzzy matching.
    """
    try:
        # List of available city names
        city_list = airport_df['city'].tolist()

        # Perform fuzzy matching
        closest_match, score, _ = process.extractOne(
            user_city.strip().title(), city_list, scorer=fuzz.ratio
        )

        # Debugging: Print matching details
        print(f"Input: {user_city}, Closest Match: {closest_match}, Score: {score}")

        # Threshold to ensure it's a valid match
        if score >= 50:  # 70 is a lenient similarity threshold
            return closest_match
        else:
            return None
    except Exception as e:
        print(f"Error finding closest city: {e}")
        return None


# get_closest_city('Hydrabad')

In [ ]:
def get_airport_code(user_city):
    """
    Get the airport code for the given city.
    """
    try:
        # Find the closest matching city
        closest_city = get_closest_city(user_city)

        if closest_city:
            # Filter the DataFrame to find the airport code for the closest city
            result = airport_df[airport_df['city'] == closest_city]['airport_code']
            return result.iloc[0] if not result.empty else None
        else:
            return None
    except Exception as e:
        print(f"Error fetching airport code for {user_city}: {e}")
        return None
    
    
# get_airport_code("Bumbai")

Input: Bumbai, Closest Match: Mumbai, Score: 83.33333333333334


'BOM'

In [10]:
def get_railway_station_code(user_city):
    """
    Get the airport code for the given city.
    """
    try:
        # Find the closest matching city
        closest_city = get_closest_city(user_city)

        if closest_city:
            # Filter the DataFrame to find the airport code for the closest city
            result = airport_df[airport_df['city'] == closest_city]['railway_station_code']
            return result.iloc[0] if not result.empty else None
        else:
            return None
    except Exception as e:
        print(f"Error fetching airport code for {user_city}: {e}")
        return None
    
    
get_railway_station_code("Bumbai")

Input: Bumbai, Closest Match: Mumbai, Score: 83.33333333333334


'CSMT'

In [ ]:
import requests
import json
from datetime import datetime

# Function to get user input and format the date
def get_user_input():
    
    # Taking User Input
    source = input("Enter source station code: ").strip().upper()
    destination = input("Enter destination station code: ").strip().upper()
    
    
    # Fetching Station Code
    srcCode = get_railway_station_code(source)
    destCode = get_railway_station_code(destination)
    travel_date = input("Enter travel date (YYYY-MM-DD): ").strip()
    
    # Format the date as required by the API (YYYYMMDD)
    formatted_date = datetime.strptime(travel_date, "%Y-%m-%d").strftime("%Y%m%d")
    return srcCode, destCode, formatted_date

# Main function to fetch train details
def fetch_train_details():
    # Get user input
    source, destination, formatted_date = get_user_input()

    # Construct API URL
    url = f"https://railways.makemytrip.com/api/tbsWithAvailabilityAndRecommendation/{source}/{destination}/{formatted_date}"

    # Headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    try:
        # Make a GET request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Parse JSON response
        data = response.json()

        # Extract train details
        trains = data.get("trainBtwnStnsList", [])
        if not trains:
            print("No trains found for the given input.")
            return

        extracted_trains = []
        for train in trains:
            # Extract ticket prices for specific classes
            ticket_prices = {availability.get("className", "N/A"): availability.get("totalFare", "N/A") for availability in train.get("tbsAvailability", [])}

            # Train information
            train_info = {
                "Train Number": train.get("trainNumber"),
                "Train Name": train.get("trainName"),
                "Source Station Code": train.get("frmStnCode"),
                "Source City": train.get("frmStnCity"),
                "Destination Station Code": train.get("toStnCode"),
                "Destination City": train.get("toStnCity"),
                "Travel Date": formatted_date,
                "Ticket Prices": {class_name: ticket_prices.get(class_name, "Not Available") for class_name in ["SL", "3A", "2A", "1A"]},
            }
            extracted_trains.append(train_info)

        # Print extracted details
        print(json.dumps(extracted_trains, indent=4))

    except requests.exceptions.RequestException as e:
        print(f"HTTP Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Run the function
if __name__ == "__main__":
    fetch_train_details()

Input: NAGPOOR, Closest Match: Nagpur, Score: 76.92307692307692
Input: RIAPUR, Closest Match: Raipur, Score: 83.33333333333334
[
    {
        "Train Number": "12771",
        "Train Name": "Sc R Sf Express",
        "Source Station Code": "NGP",
        "Source City": "Nagpur",
        "Destination Station Code": "R",
        "Destination City": "Raipur",
        "Travel Date": "20241214",
        "Ticket Prices": {
            "SL": 240,
            "3A": 610,
            "2A": 840,
            "1A": "Not Available"
        }
    },
    {
        "Train Number": "18238",
        "Train Name": "Chhatisgarh Exp",
        "Source Station Code": "NGP",
        "Source City": "Nagpur",
        "Destination Station Code": "R",
        "Destination City": "Raipur",
        "Travel Date": "20241214",
        "Ticket Prices": {
            "SL": 210,
            "3A": 560,
            "2A": 795,
            "1A": 1315
        }
    },
    {
        "Train Number": "12129",
        "Train Name

In [27]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.1/943.1 kB 8.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import http.client
import json
from datetime import datetime

# Function to get user input and format the date
def get_user_input():
    # Taking User Input
    source = input("Enter source station code: ").strip().upper()
    destination = input("Enter destination station code: ").strip().upper()
    travel_date = input("Enter travel date (YYYY-MM-DD): ").strip()

    # Get Airport code
    srcCode= get_airport_code(source)
    dstCode= get_airport_code(destination)
    
    # Format the date as required by the API
    formatted_date = datetime.strptime(travel_date, "%Y-%m-%d").strftime("%Y-%m-%d")
    return srcCode, dstCode, formatted_date

# Function to fetch flights data
def get_skyScanner_flights(source, destination, formatted_date):
    
    # API connection setup
    conn = http.client.HTTPSConnection("sky-scanner3.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': "3254875c10mshf948cf35f09d589p1c7181jsn2faeb668170d",
        'x-rapidapi-host': "sky-scanner3.p.rapidapi.com"
    }

    # Construct API endpoint
    endpoint = f"/flights/search-one-way?fromEntityId={source}&toEntityId={destination}&departDate={formatted_date}&currency=INR&cabinClass=economy"
    
    # API request
    conn.request("GET", endpoint, headers=headers)
    res = conn.getresponse()
    data = res.read()

    # Parse the API response
    response = json.loads(data.decode("utf-8"))

    # Extract details
    itineraries = response.get("data", {}).get("itineraries", [])
    flights = []

    for itinerary in itineraries:
        price = itinerary.get("price", {}).get("raw", "N/A")
        legs = itinerary.get("legs", [])
        
        for leg in legs:
            flight_details = {
                "Airlines": leg.get("carriers", {}).get("marketing", [{}])[0].get("name", "N/A"),
                "Source City": leg.get("origin", {}).get("city", "N/A"),
                "Source Airport Name": leg.get("origin", {}).get("name", "N/A"),
                "Destination City": leg.get("destination", {}).get("city", "N/A"),
                "Destination Airport Name": leg.get("destination", {}).get("name", "N/A"),
                "Departure Date": leg.get("departure", "").split("T")[0],
                "Departure Time": leg.get("departure", "").split("T")[1] if "T" in leg.get("departure", "") else "N/A",
                "Arrival Date": leg.get("arrival", "").split("T")[0],
                "Arrival Time": leg.get("arrival", "").split("T")[1] if "T" in leg.get("arrival", "") else "N/A",
                "Duration of Travel": f"{leg.get('durationInMinutes', 'N/A')} minutes",
                "Flight Number": leg.get("segments", [{}])[0].get("flightNumber", "N/A"),
                "Price in INR": f"₹ {price:.2f}" if price != "N/A" else "N/A"
            }
            flights.append(flight_details)

    # Print the extracted details
    for flight in flights:
        print(flight)
        print("=" * 50)

# Main execution
if __name__ == "__main__":
    source, destination, formatted_date = get_user_input()
    get_skyScanner_flights(source, destination, formatted_date)

Input: BUMBAE, Closest Match: Mumbai, Score: 66.66666666666667
Input: DELLI, Closest Match: New Delhi, Score: 57.14285714285714
{'Airlines': 'IndiGo', 'Source City': 'Mumbai', 'Source Airport Name': 'Mumbai', 'Destination City': 'New Delhi', 'Destination Airport Name': 'Indira Gandhi International ', 'Departure Date': '2024-12-19', 'Departure Time': '06:00:00', 'Arrival Date': '2024-12-19', 'Arrival Time': '08:05:00', 'Duration of Travel': '125 minutes', 'Flight Number': '2156', 'Price in INR': '₹ 5814.73'}
{'Airlines': 'IndiGo', 'Source City': 'Mumbai', 'Source Airport Name': 'Mumbai', 'Destination City': 'New Delhi', 'Destination Airport Name': 'Indira Gandhi International ', 'Departure Date': '2024-12-19', 'Departure Time': '13:05:00', 'Arrival Date': '2024-12-19', 'Arrival Time': '15:10:00', 'Duration of Travel': '125 minutes', 'Flight Number': '615', 'Price in INR': '₹ 6870.80'}
{'Airlines': 'Air India Express', 'Source City': 'Mumbai', 'Source Airport Name': 'Mumbai', 'Destinatio

In [ ]:
import http.client
import json
from datetime import datetime

# Function to get user input and format the date
def get_user_input():
    # Taking User Input
    source = input("Enter source station code or city: ").strip().upper()
    destination = input("Enter destination station code or city: ").strip().upper()
    travel_date = input("Enter travel date (YYYY-MM-DD): ").strip()

    # Get Airport code
    srcCode = get_airport_code(source)
    destCode = get_airport_code(destination)
    
    # Format the date as required by the API
    formatted_date = datetime.strptime(travel_date, "%Y-%m-%d").strftime("%Y-%m-%d")
    return srcCode, destCode, formatted_date

# Function to fetch flights data
def get_PriceLine_flights(source, destination, formatted_date):
    
    # API connection setup
    conn = http.client.HTTPSConnection("priceline-com2.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': "3254875c10mshf948cf35f09d589p1c7181jsn2faeb668170d",
        'x-rapidapi-host': "priceline-com2.p.rapidapi.com"
    }
    
    # API request
    endpoint = f"/flights/search-one-way?originAirportCode={source}&destinationAirportCode={destination}&departureDate={formatted_date}"
    conn.request("GET", endpoint, headers=headers)
    res = conn.getresponse()
    data = res.read()

    # Parse the API response
    try:
        response = json.loads(data.decode("utf-8"))
    except json.JSONDecodeError:
        print("Error decoding the API response.")
        return

    # Extract details
    listings = response.get("data", {}).get("listings", [])
    if not listings:
        print("No flight listings found.")
        return

    flights = []

    for listing in listings:
        slices = listing.get("slices", [])
        price_usd = listing.get("totalPriceWithDecimal", {}).get("price", "N/A")
        try:
            price_inr = float(price_usd) * 84.46 if price_usd != "N/A" else "N/A"
        except ValueError:
            price_inr = "N/A"

        for slice_data in slices:
            segments = slice_data.get("segments", [])
            total_duration = slice_data.get("durationInMinutes", "N/A")

            for segment in segments:
                flight_details = {
                    "Airlines": segment.get("marketingAirline", "N/A"),
                    "Flight Number": segment.get("flightNumber", "N/A"),
                    "Source City": segment.get("departInfo", {}).get("airport", {}).get("name", "N/A"),
                    "Source Airport Code": segment.get("departInfo", {}).get("airport", {}).get("code", "N/A"),
                    "Destination City": segment.get("arrivalInfo", {}).get("airport", {}).get("name", "N/A"),
                    "Destination Airport Code": segment.get("arrivalInfo", {}).get("airport", {}).get("code", "N/A"),
                    "Departure Date": segment.get("departInfo", {}).get("time", {}).get("dateTime", "").split("T")[0],
                    "Departure Time": segment.get("departInfo", {}).get("time", {}).get("dateTime", "").split("T")[1] if "T" in segment.get("departInfo", {}).get("time", {}).get("dateTime", "") else "N/A",
                    "Arrival Date": segment.get("arrivalInfo", {}).get("time", {}).get("dateTime", "").split("T")[0],
                    "Arrival Time": segment.get("arrivalInfo", {}).get("time", {}).get("dateTime", "").split("T")[1] if "T" in segment.get("arrivalInfo", {}).get("time", {}).get("dateTime", "") else "N/A",
                    "Duration of Travel": f"{segment.get('duration', 'N/A')} minutes",
                    "Stop Quantity": segment.get("stopQuantity", 0),
                    "Equipment Name": segment.get("equipmentName", "N/A"),
                    "Price in INR": f"₹ {price_inr:.2f}" if price_inr != "N/A" else "N/A"
                }
                flights.append(flight_details)

    # Print the extracted details
    if flights:
        for flight in flights:
            print(flight)
            print("=" * 50)
    else:
        print("No flights found.")

# Main execution
if __name__ == "__main__":
    source, destination, formatted_date = get_user_input()
    get_PriceLine_flights(source, destination, formatted_date)

Input: BUMBAE, Closest Match: Mumbai, Score: 66.66666666666667
Input: DELLI, Closest Match: New Delhi, Score: 57.14285714285714
{'Airlines': '6E', 'Flight Number': '534', 'Source City': 'Chhatrapati Shivaji Maharaj Intl Airport', 'Source Airport Code': 'BOM', 'Destination City': 'Sardar Vallabhbhai Patel Airport', 'Destination Airport Code': 'AMD', 'Departure Date': '2024-12-19', 'Departure Time': '04:30:00', 'Arrival Date': '2024-12-19', 'Arrival Time': '05:35:00', 'Duration of Travel': '65 minutes', 'Stop Quantity': 0, 'Equipment Name': 'Airbus A320', 'Price in INR': '₹ 6380.11'}
{'Airlines': '6E', 'Flight Number': '2378', 'Source City': 'Sardar Vallabhbhai Patel Airport', 'Source Airport Code': 'AMD', 'Destination City': 'Indira Gandhi Intl Airport', 'Destination Airport Code': 'DEL', 'Departure Date': '2024-12-19', 'Departure Time': '07:05:00', 'Arrival Date': '2024-12-19', 'Arrival Time': '08:45:00', 'Duration of Travel': '100 minutes', 'Stop Quantity': 0, 'Equipment Name': 'Airbu

In [ ]:
import http.client
import json
from datetime import datetime

# Function to get user input and format the date
def get_user_input():
    # Taking User Input
    source = input("Enter source station code or city: ").strip().upper()
    destination = input("Enter destination station code or city: ").strip().upper()
    travel_date = input("Enter travel date (YYYY-MM-DD): ").strip()

    # Get Airport code
    srcCode = get_airport_code(source)
    destCode = get_airport_code(destination)
    
    # Format the date as required by the API
    formatted_date = datetime.strptime(travel_date, "%Y-%m-%d").strftime("%Y-%m-%d")
    return srcCode, destCode, formatted_date

# Function to fetch flights data
def get_tripadvisor_flights(source, destination, formatted_date):
    # API connection setup
    conn = http.client.HTTPSConnection("tripadvisor16.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': "3254875c10mshf948cf35f09d589p1c7181jsn2faeb668170d",
        'x-rapidapi-host': "tripadvisor16.p.rapidapi.com"
    }
    # API request
    endpoint = f"/api/v1/flights/searchFlights?sourceAirportCode={source}&destinationAirportCode={destination}&date={formatted_date}&itineraryType=ONE_WAY&sortOrder=ML_BEST_VALUE&numAdults=1&numSeniors=0&classOfService=ECONOMY&pageNumber=1&nearby=yes&nonstop=yes"
    conn.request("GET", endpoint, headers=headers)
    res = conn.getresponse()
    data = res.read()

    # Parse the API response
    try:
        response = json.loads(data.decode("utf-8"))
    except json.JSONDecodeError:
        print("Error decoding the API response.")
        return

    # Extract details
    flights_data = response.get("data", {}).get("flights", [])
    if not flights_data:
        print("No flight data available.")
        return

    flights = []

    for flight in flights_data:
        purchase_links = flight.get("purchaseLinks", [])
        price_usd = purchase_links[0].get("totalPrice", "N/A") if purchase_links else "N/A"
        try:
            price_inr = float(price_usd) * 84.46 if price_usd != "N/A" else "N/A"
        except ValueError:
            price_inr = "N/A"

        for segment in flight.get("segments", []):
            for leg in segment.get("legs", []):
                flight_details = {
                    "Airline Name": leg.get("marketingCarrier", {}).get("displayName", "N/A"),
                    "Source City/Airport Code": leg.get("originStationCode", "N/A"),
                    "Destination City/Airport Code": leg.get("destinationStationCode", "N/A"),
                    "Departure Date/Time": leg.get("departureDateTime", "N/A"),
                    "Arrival Date/Time": leg.get("arrivalDateTime", "N/A"),
                    "Flight Class": leg.get("classOfService", "N/A"),
                    "Flight Number": leg.get("flightNumber", "N/A"),
                    "Number of Stops": leg.get("numStops", "N/A"),
                    "Distance (KM)": leg.get("distanceInKM", "N/A"),
                    "Price (INR)": f"₹ {price_inr:.2f}" if price_inr != "N/A" else "N/A"
                }
                flights.append(flight_details)

    # Print the extracted details
    if flights:
        for flight in flights:
            print(flight)
            print("=" * 50)
    else:
        print("No flights found.")

# Main execution
if __name__ == "__main__":
    source, destination, formatted_date = get_user_input()
    get_tripadvisor_flights(source, destination, formatted_date)

Input: VISHAKAPTNAM, Closest Match: Vishakhapatnam, Score: 92.3076923076923
Input: MUMBAE, Closest Match: Mumbai, Score: 83.33333333333334
{'Airline Name': 'IndiGo', 'Source City/Airport Code': 'VTZ', 'Destination City/Airport Code': 'BOM', 'Departure Date/Time': '2024-12-19T15:10:00+05:30', 'Arrival Date/Time': '2024-12-19T17:20:00+05:30', 'Flight Class': 'ECONOMY', 'Flight Number': 5247, 'Number of Stops': 0, 'Distance (KM)': 1104.5164, 'Price (INR)': '₹ 5571.83'}
{'Airline Name': 'Air India', 'Source City/Airport Code': 'VTZ', 'Destination City/Airport Code': 'BOM', 'Departure Date/Time': '2024-12-19T16:10:00+05:30', 'Arrival Date/Time': '2024-12-19T18:35:00+05:30', 'Flight Class': 'ECONOMY', 'Flight Number': 2460, 'Number of Stops': 0, 'Distance (KM)': 1104.5164, 'Price (INR)': '₹ 10974.73'}


In [12]:
import psycopg2
import requests
from datetime import datetime, timedelta
import json

In [ ]:
# Function to fetch bus data from the Zingbus API
def fetch_bus_data(from_city, to_city, trip_date):
    url = f"https://www.zingbus.com/v1/search/zingbus/buses/?fromCity={from_city}&toCity={to_city}&tripDate={trip_date}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            raw_content = response.text
            json_objects = raw_content.split('\n')  # Assuming JSON objects are separated by newlines
            parsed_data = [json.loads(obj) for obj in json_objects if obj.strip()]
            return parsed_data  # Return a list of parsed JSON objects
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            print(f"Response content: {response.text}")  # Print the error content from the API
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        return None

Failed to fetch data. Status code: 400
Response content: {"statusCode":"error","details":"No buses available"}
No data to insert.


In [14]:
# Function to convert Unix timestamp to IST (Indian Standard Time)
def convert_timestamp(timestamp_ms):
    # Convert milliseconds to seconds
    timestamp_s = timestamp_ms / 1000
    # Convert to a datetime object in UTC
    utc_time = datetime.utcfromtimestamp(timestamp_s)
    # Add 5 hours and 30 minutes to convert to IST
    ist_time = utc_time + timedelta(hours=5, minutes=30)
    # Return the time in the format: 'YYYY-MM-DD HH:MM:SS'
    return ist_time.strftime('%Y-%m-%d %H:%M:%S')

In [17]:
import psycopg2
from psycopg2 import sql

def convert_timestamp(milliseconds):
    from datetime import datetime, timezone, timedelta
    ist = timezone(timedelta(hours=5, minutes=30))
    return datetime.fromtimestamp(milliseconds / 1000, tz=ist).strftime('%Y-%m-%d %H:%M:%S')

def insert_bus_data(bus_data):
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="BUS_DATA",  # Change this to your database name
        user="postgres",  # Replace with your PostgreSQL username
        password="2301",  # Replace with your PostgreSQL password
        host="192.168.42.113",  # Or your host (use 'localhost' if running locally)
        port="5432"  # Default port for PostgreSQL
    )

    # Create a cursor object to interact with the database
    cur = conn.cursor()

    # Create the table if it doesn't exist
    create_table_query = """
        CREATE TABLE IF NOT EXISTS buses (
            id SERIAL PRIMARY KEY,
            source_city VARCHAR(255) NOT NULL,
            destination_city VARCHAR(255) NOT NULL,
            bus_type VARCHAR(255),
            departure_time TIMESTAMP NOT NULL,
            arrival_time TIMESTAMP NOT NULL,
            total_travel_time VARCHAR(255),
            fare NUMERIC
        )
    """
    cur.execute(create_table_query)

    # Loop through the list of JSON objects
    for data in bus_data:  # bus_data is now a list of dictionaries
        # Loop through each trip and insert data into the PostgreSQL table
        trips = data.get("trips", [])
        for trip in trips:
            # Extract source and destination cities from each trip
            source_city = trip.get("fromCity", "")
            destination_city = trip.get("toCity", "")

            # Debug: print the extracted cities to confirm they are being pulled
            print(f"Source City: {source_city}, Destination City: {destination_city}")

            # Convert the departure and arrival times to IST
            departure_time_ist = convert_timestamp(trip.get("startTimeInMills", 0))
            arrival_time_ist = convert_timestamp(trip.get("endTimeInMills", 0))

            bus_info = {
                "source_city": source_city,
                "destination_city": destination_city,
                "bus_type": trip.get("type", ""),
                "departure_time": departure_time_ist,
                "arrival_time": arrival_time_ist,
                "total_travel_time": trip.get("timeDifference", ""),
                "fare": trip.get("fare", ""),
            }

            # Prepare the SQL INSERT statement (without date_time)
            insert_query = """
                INSERT INTO buses (source_city, destination_city, bus_type, departure_time, arrival_time, total_travel_time, fare)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """

            # Execute the query with the bus_info values (without date_time)
            cur.execute(insert_query, (
                bus_info['source_city'], 
                bus_info['destination_city'], 
                bus_info['bus_type'], 
                bus_info['departure_time'], 
                bus_info['arrival_time'], 
                bus_info['total_travel_time'], 
                bus_info['fare'], 
            ))

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

    print("Data inserted successfully into PostgreSQL database.")

In [18]:
from_city = input("Enter source city (e.g., Delhi): ")
to_city = input("Enter destination city (e.g., Manali): ")
trip_date = input("Enter trip date (YYYY-MM-DD): ")

# Fetch bus data from the API
bus_data = fetch_bus_data(from_city, to_city, trip_date)

if bus_data:
    insert_bus_data(bus_data)

Source City: Delhi, Destination City: Manali
Source City: Delhi, Destination City: Manali
Source City: Delhi, Destination City: Manali
Source City: Delhi, Destination City: Manali
Source City: Delhi, Destination City: Manali
Source City: Delhi, Destination City: Manali
Data inserted successfully into PostgreSQL database.
